# Connect to Coinbase Cloud REST API and Visualise Some Stuff



## Overview

- Connect to the public coinbase.com REST API via Python
- Load data from Coinbase into Pandas
- Add some useful crypto trading indicators to the data
- Visualise Bitcoin (and/or ETH, SOL etc.) historic data in Python

Coinbase Cloud API can be found [here](https://docs.cloud.coinbase.com/exchange/reference/). 
Using the following endpoints:
- https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproducts
- https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles
- https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductstats

## Step 0 - Requirements

- Install all necessary libraries:

````
!pip install -r requirements.txt 
````


## Step 1 - Imports, Libraries and Functions

We need the following Python imports:

- **Requests:** For connecting to the REST endpoints from Coinbase.
- **Pandas** 
- **Json** 

In [1]:

import pandas as pd
import json
import requests
from datetime import datetime, timedelta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.error import HTTPError


In [2]:
# Choose currency that should be used to display data
FIAT_CURRENCIES = ["EUR"]#Changed the currency to EUR to test 
MY_QUOTE_CURRENCY = FIAT_CURRENCIES[0]

# I am only interested in a few currencies that I want to trade, so let's add them here:
MY_CRYPTO_CURRENCIES = ["BTC","ETH","SOL","XRP"] #added the XRP
GRANULARITIES = ['DAILY','60MIN','15MIN','1MIN'] 

CANDLE_INCREASE_COLOR = 'blue'
CANDLE_DECREASE_COLOR = 'red'
MACD_COLOR = '#A9B1F5'
MACD_SIGNAL_COLOR = '#FFD1A3'
RED_COLOR = '#D61170'
GREEN_COLOR = '#3FFA68'
#PLOTLY_TEMPLATE = 'plotly_dark'
PLOTLY_TEMPLATE = 'plotly'

In [3]:
# Reusable method that will take an endpoint and tries to make a connection. Returns the response.
def connect(url, *args, **kwargs):  # Define a function called 'connect' that takes a URL and additional arguments (optional ones).
    try:  # Start a try block to attempt to do something that might cause an error.
        if kwargs.get('param', None) is not None:  # Check if there are parameters ('param') passed in the kwargs (optional arguments).
            params = kwargs.get('param')  # If parameters are provided, store them in the 'params' variable.
            response = requests.get(url, params)  # Make an HTTP GET request to the URL with the parameters.
        else:  # If no parameters are provided.
            response = requests.get(url)  # Make an HTTP GET request to the URL without parameters.
        response.raise_for_status()  # Check if the response status code is successful (2xx). If not, raise an HTTPError.
        print(f'HTTP connection {url} successful!')  # If no errors occur, print a success message with the URL.
        return response  # Return the response from the HTTP request.
    except HTTPError as http_err:  # Catch any HTTP-related errors.
        print(f'HTTP error occurred: {http_err}')  # Print the error message for HTTP-related errors.
    except Exception as err:  # Catch any other type of errors that might occur.
        print(f'Other error occurred: {err}')  # Print the error message for other types of errors.

# method that establishes a connection to Coinbase and will retrieve historic data for the chosen interval
def get_historic_data(start_date, end_date, interval, base_currency, quote_currency):
    # Define the valid intervals for data fetching (like DAILY, 60MIN, etc.)
    VALID_INTERVAL = {'DAILY', '60MIN', '15MIN', '5MIN', '1MIN'}

    # Check if the given interval is valid. If not, it raises an error.
    if interval not in VALID_INTERVAL:
        raise ValueError("results: interval must be one of %r." % VALID_INTERVAL)

    # Based on the selected interval, map it to the corresponding granularity (time frame in seconds)
    if interval == '1MIN':  # If the interval is '1MIN', set granularity to 60 seconds.
        granularity = '60'  
    elif interval == '15MIN':  # If the interval is '15MIN', set granularity to 900 seconds (15 minutes).
        granularity = '900'
    elif interval == '5MIN':  # If the interval is '5MIN', set granularity to 300 seconds (5 minutes).
        granularity = '300'
    elif interval == '60MIN':  # If the interval is '60MIN', set granularity to 3600 seconds (60 minutes).
        granularity = '3600'   
    else:  # If the interval is 'DAILY', set granularity to 86400 seconds (1 day).
        granularity = '86400'

    # Prepare the parameters needed for the API request (start date, end date, and granularity).
    params = {'start':start_date, 'end':end_date, 'granularity':granularity}

    # Call the 'connect' function with the constructed URL and parameters to get data from Coinbase API.
    data = json.loads(connect('https://api.exchange.coinbase.com/products/'+quote_currency+'-'+base_currency+'/candles', param = params).text)  
    # The URL is for the Coinbase API to get candlestick data. 
    # The 'quote_currency' and 'base_currency' are swapped to ensure the pair is in base-quote format like BTC-EUR instead of EUR-BTC.
    # The response is returned as JSON for easy human reading and converted to a Python list.

    # Add the quote currency (EUR) to each data entry to keep track of it.
    [x.append(quote_currency) for x in data]

    # Add the interval (DAILY, 60MIN...) to each data entry to keep track of it.
    [x.append(interval) for x in data]

    # Return the modified data with additional details about the currency and interval.
    return data

Before we start we will retrieve the time on the server and compare it to the client time:

In [4]:
# Server time is fetched by making a GET request to Coinbase's 'time' endpoint
server_time = json.loads(connect('https://api.exchange.coinbase.com/time').text)
# The 'connect' function fetches the server time from the Coinbase API. 
# The response is then converted from JSON format to a Python dictionary using json.loads.

# Server time does not comply to iso format(yy/mm/dd), therefore slight modification of string needed
# The 'iso' key from the server's response contains the time in a format like '2025-03-07T12:45:30Z'. 
# It needs to be slightly adjusted for Python's 'fromisoformat' method, which expects a space instead of the 'T' between date and time.
server_time_now = datetime.fromisoformat(server_time['iso'].replace('T', ' ', 1)[0:19])
# Replace the 'T' with a space (just once) to match the expected ISO format, 
# then take the first 19 characters (to remove the 'Z' at the end, which signifies UTC timezone) 
# and convert it to a datetime object using 'fromisoformat'.

# Display the server time as a datetime object
print(f'Time on server: {server_time_now}')
# Print the time received from the server.

# Display the current client time
print(f'Time on client: {datetime.now()}')
# Print the current time on the client (the computer running the code).


HTTP connection https://api.exchange.coinbase.com/time successful!
Time on server: 2026-01-22 20:43:34
Time on client: 2026-01-22 21:43:35.116223


## Step 2 - Load Available Coinbase Products (Crypto Currencies)

Let's check out which currencies and trading pairs are available. You can find this information under the `products` endpoint. 

In [5]:
# This sends a GET request to the Coinbase API to get the list of trading pairs(products).
response = connect('https://api.exchange.coinbase.com/products')

# This stores the binary content (raw response data) from the API.
response_content = response.content

# This stores the API response as a string so we can read and parse it as JSON.
response_text = response.text

# This stores metadata from the response, like id, display name, status, max slippage percentage, etc.
response_headers = response.headers

# We convert the JSON response into a Pandas DataFrame like a table with rows and columns for easier manipulation.
df_products = pd.read_json(response_text)

# Display the number of columns and rows in the DataFrame.
print("\nNumber of columns in the dataframe: %i" % (df_products.shape[1]))
print("Number of rows in the dataframe: %i\n" % (df_products.shape[0]))

# Convert the column names into a list to check them (useful for reference).
columns = list(df_products.columns)

# Sort the DataFrame by the 'id' column to organize the data.
df_products.sort_values('id', inplace=True)

# Create a copy of the DataFrame to work on without modifying the original.
df_products_formatted = df_products.copy()

# Apply formatting to the 'base_increment' column to show it with 9 decimal places.
df_products_formatted['base_increment'] = df_products_formatted['base_increment'].apply(lambda x: "{:,.9f}".format((x)))

# Filter the DataFrame to show only the rows that are related to the currencies we're interested in.
# We combine the currency pairs from MY_CRYPTO_CURRENCIES and MY_QUOTE_CURRENCY to create the matching pairs.
df_products_formatted[df_products_formatted.id.isin(list(map(lambda orig_string: orig_string+'-'+MY_QUOTE_CURRENCY, MY_CRYPTO_CURRENCIES)))][['id', 'base_currency', 'quote_currency', 'base_increment', 'max_slippage_percentage']]#added the max slippage percentage

HTTP connection https://api.exchange.coinbase.com/products successful!

Number of columns in the dataframe: 18
Number of rows in the dataframe: 776



C:\Users\hp\AppData\Local\Temp\ipykernel_17344\3215380966.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_products = pd.read_json(response_text)


,id,base_currency,quote_currency,base_increment,max_slippage_percentage
471,BTC-EUR,BTC,EUR,0.000000010,0.03
363,ETH-EUR,ETH,EUR,0.000000010,0.03
699,SOL-EUR,SOL,EUR,0.001000000,0.03
506,XRP-EUR,XRP,EUR,0.000001000,0.05


## Step 3 - Load 24h Stats

Basic stats from the last 24 hours for each of the crypto currencies that we have defined in our list above. 

In [6]:
# Create an empty list to store data for each cryptocurrency pair.
currency_rows = []

# Loop through each cryptocurrency in MY_CRYPTO_CURRENCIES.
for currency in MY_CRYPTO_CURRENCIES:
    # Send a GET request to the Coinbase API to get the 24-hour stats for the products aka trading/pairs.
    data = json.loads(connect('https://api.exchange.coinbase.com/products/'+currency+'-'+MY_QUOTE_CURRENCY+'/stats').text)
    # Append the data for each currency pair into the currency_rows list.
    currency_rows.append(data)

# Create a DataFrame from the currency_rows list with the cryptocurrency names as the index.
df_24hstats = pd.DataFrame(currency_rows, index = MY_CRYPTO_CURRENCIES)

# Add columns for base_currency and quote_currency.
df_24hstats['base_currency'] = df_24hstats.index
df_24hstats['quote_currency'] = MY_QUOTE_CURRENCY

# Convert the columns with numeric data into the appropriate type (float).
df_24hstats['open'] = df_24hstats['open'].astype(float)
df_24hstats['high'] = df_24hstats['high'].astype(float)
df_24hstats['low'] = df_24hstats['low'].astype(float)
df_24hstats['volume'] = df_24hstats['volume'].astype(float)
df_24hstats['last'] = df_24hstats['last'].astype(float)
df_24hstats['volume_30day'] = df_24hstats['volume_30day'].astype(float)

# Calculate the performance (percentage change) for each cryptocurrency in the last 24 hours.
df_24hstats['performance'] = ((df_24hstats['last']-df_24hstats['open']) / df_24hstats['open']) * 100

# Create a copy of the DataFrame to format the values for display.
df_24hstats_formatted = df_24hstats.copy()

# Format the 'performance' column to show the percentage with two decimal places.
df_24hstats_formatted['performance'] = df_24hstats_formatted['performance'].apply(lambda x: "{:.2f}%".format((x)))

# Format the numerical columns ('open', 'high', 'low', 'last') to display them in euros with two decimal places.
df_24hstats_formatted['open'] = df_24hstats_formatted['open'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['high'] = df_24hstats_formatted['high'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['low'] = df_24hstats_formatted['low'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['last'] = df_24hstats_formatted['last'].apply(lambda x: "{:,.2f}€".format((x)))

# Print the formatted performance statistics.
print('Performance within last 24 hours in ' + MY_QUOTE_CURRENCY + ':')
print(df_24hstats_formatted)

HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/XRP-EUR/stats successful!
Performance within last 24 hours in EUR:
           open        high         low        last        volume  \
BTC  77,149.50€  77,320.29€  75,422.93€  76,085.25€  1.665156e+02   
ETH   2,592.28€   2,604.38€   2,477.00€   2,493.67€  1.858780e+03   
SOL     112.15€     113.00€     108.02€     109.19€  1.896374e+04   
XRP       1.69€       1.70€       1.62€       1.63€  1.438916e+06   

     volume_30day rfq_volume_24hour rfq_volume_30day base_currency  \
BTC  6.488248e+03         22.570991       295.945614           BTC   
ETH  6.167376e+04        549.099294      9145.419275           ETH   
SOL  7.406926e+05       6750.916673    106287.278925           

## Step 4 - Load Historic Data

Download data using four different time slices and periods:
- Daily data of the last 90 days
- Hourly data of the last 300 hours
- 15-minute data of the last 75 hours
- 1-minute data of the last 2 hours

All this data is stored in a pandas dataframe called df_history.

CHANGE THIS TO RETREIVE MORE DATA!

In [7]:
# Initialize an empty list to store historical data for each cryptocurrency
currency_history_rows = []

# Loop through each cryptocurrency in MY_CRYPTO_CURRENCIES
for currency in MY_CRYPTO_CURRENCIES:
    # Set the end date to the current server time in ISO format
    end_date = server_time_now.isoformat()

    # Collect data for different time intervals
    # 1-minute data (past 2 hours)
    start_date = (server_time_now - timedelta(hours=2)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, '1MIN', MY_QUOTE_CURRENCY, currency))
    
    # 15-minute data (past 75 hours)
    start_date = (server_time_now - timedelta(hours=75)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, '15MIN', MY_QUOTE_CURRENCY, currency))
    
    # 60-minute data (past 300 hours or 12.5 days)
    start_date = (server_time_now - timedelta(hours=300)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, '60MIN', MY_QUOTE_CURRENCY, currency))
    
    # Daily data (past 90 days)
    start_date = (server_time_now - timedelta(days=90)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, 'DAILY', MY_QUOTE_CURRENCY, currency))

# Create a DataFrame from the collected historical data
df_history = pd.DataFrame(currency_history_rows)

# Rename the columns based on the Coinbase API documentation
df_history.columns = ['time', 'low', 'high', 'open', 'close', 'volume', 'base_currency', 'granularity']

# Add additional columns for better readability
df_history['quote_currency'] = MY_QUOTE_CURRENCY  # Set the quote currency (EUR)
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')  # Convert time (Unix) to datetime
df_history['year'] = pd.DatetimeIndex(df_history['date']).year  # Extract year from the date
df_history['month'] = pd.DatetimeIndex(df_history['date']).month  # Extract month from the date
df_history['day'] = pd.DatetimeIndex(df_history['date']).day  # Extract day from the date
df_history['hour'] = pd.DatetimeIndex(df_history['date']).hour  # Extract hour from the date
df_history['minute'] = pd.DatetimeIndex(df_history['date']).minute  # Extract minute from the date


HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/candles successful!
HTTP connection https://api.exchange.coinba

In [8]:
print(f'Daily data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'DAILY\'')

Daily data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
719,1769040000,75422.9300,77304.2100,76593.6000,76111.0300,1.463944e+02,BTC,DAILY,EUR,2026-01-22,2026,1,22,0,0
720,1768953600,74480.0000,77364.8600,75331.1400,76590.1500,3.095323e+02,BTC,DAILY,EUR,2026-01-21,2026,1,21,0,0
721,1768867200,74910.0000,79763.0100,79544.5400,75323.0500,3.620260e+02,BTC,DAILY,EUR,2026-01-20,2026,1,20,0,0
722,1768780800,79104.3500,80512.7400,80512.7400,79525.9100,1.817849e+02,BTC,DAILY,EUR,2026-01-19,2026,1,19,0,0
723,1768694400,80514.9400,82437.2500,82074.0400,80573.8300,1.074328e+02,BTC,DAILY,EUR,2026-01-18,2026,1,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3176,1761696000,2.1875,2.2898,2.2330,2.1979,1.500593e+06,XRP,DAILY,EUR,2025-10-29,2025,10,29,0,0
3177,1761609600,2.2054,2.3122,2.2597,2.2332,1.929326e+06,XRP,DAILY,EUR,2025-10-28,2025,10,28,0,0
3178,1761523200,2.2351,2.3176,2.2749,2.2595,1.866251e+06,XRP,DAILY,EUR,2025-10-27,2025,10,27,0,0
3179,1761436800,2.2288,2.2896,2.2312,2.2741,1.574613e+06,XRP,DAILY,EUR,2025-10-26,2025,10,26,0,0


In [9]:
print(f'60 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'60MIN\'').tail(5)

60 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
3086,1768050000,1.7963,1.8051,1.8051,1.7969,41112.973417,XRP,60MIN,EUR,2026-01-10 13:00:00,2026,1,10,13,0
3087,1768046400,1.8045,1.8131,1.8071,1.8049,51855.432369,XRP,60MIN,EUR,2026-01-10 12:00:00,2026,1,10,12,0
3088,1768042800,1.8036,1.8085,1.8082,1.8069,20351.933874,XRP,60MIN,EUR,2026-01-10 11:00:00,2026,1,10,11,0
3089,1768039200,1.8000,1.8082,1.8011,1.8082,36777.487684,XRP,60MIN,EUR,2026-01-10 10:00:00,2026,1,10,10,0
3090,1768035600,1.7965,1.8023,1.7976,1.8007,22613.217561,XRP,60MIN,EUR,2026-01-10 09:00:00,2026,1,10,9,0


In [10]:
print(f'15 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'15MIN\'').tail(5)

15 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
2786,1768848300,1.7249,1.7299,1.7299,1.7264,8846.800088,XRP,15MIN,EUR,2026-01-19 18:45:00,2026,1,19,18,45
2787,1768847400,1.7276,1.7337,1.7279,1.7295,20614.694056,XRP,15MIN,EUR,2026-01-19 18:30:00,2026,1,19,18,30
2788,1768846500,1.7264,1.7357,1.7339,1.7277,14552.426224,XRP,15MIN,EUR,2026-01-19 18:15:00,2026,1,19,18,15
2789,1768845600,1.7324,1.7384,1.7332,1.7331,16678.332254,XRP,15MIN,EUR,2026-01-19 18:00:00,2026,1,19,18,0
2790,1768844700,1.7263,1.7341,1.7263,1.7325,22913.269677,XRP,15MIN,EUR,2026-01-19 17:45:00,2026,1,19,17,45


In [11]:
print(f'1 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'1MIN\'').tail(5)

1 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
2486,1769107860,1.6345,1.6355,1.6345,1.6355,1077.256274,XRP,1MIN,EUR,2026-01-22 18:51:00,2026,1,22,18,51
2487,1769107800,1.6355,1.6381,1.6381,1.6355,3842.056901,XRP,1MIN,EUR,2026-01-22 18:50:00,2026,1,22,18,50
2488,1769107680,1.6393,1.6400,1.6400,1.6395,11307.041467,XRP,1MIN,EUR,2026-01-22 18:48:00,2026,1,22,18,48
2489,1769107620,1.6400,1.6400,1.6400,1.6400,5744.169348,XRP,1MIN,EUR,2026-01-22 18:47:00,2026,1,22,18,47
2490,1769107500,1.6400,1.6409,1.6408,1.6400,6563.446863,XRP,1MIN,EUR,2026-01-22 18:45:00,2026,1,22,18,45


## Step 5 - Insights

Let's add a few more parameters to the basic data (available in df_history):

- **SMA3 and SMA7 and SMA12** (Simple Moving Average over last 3 and 7 time slices): This is a price based, lagging (or reactive) indicator that displays the average price of a security over a certain period of time. Moving Averages smooth out the “noise” when interpreting charts. Noise is made up of fluctuations of both price and volume. Because a Moving Average is a lagging indicator and reacts to events that have already happened, it is not used as a predictive indicator but rather an interpretive one. Simple Moving Average is an unweighted Moving Average. This means that each period in the data set has equal importance and is weighted equally. 
- **EMA12 and EMA26** (Exponential Moving Average over last 12 and 26 time slices): The EMA has much less lag than the SMA because it puts a greater importance on more recent prices. It therefore turns quicker than the SMA. You can choose different time slices, but 12 and 26 are commonly used durations.
- **MACD** (Moving Average Convergence/Divergence): A great indicator to identify the overall trend of any security (e.g. crypto currencies). It takes the difference between the short term EMA and the long term EMA. A positive MACD value is an indicator for a positive market trend. A negative MACD value is an indicator for a negative market trend. 
- **MACD Signal**: The MACD Signal line is the EMA over a certain set of time slices of the MACD line. Typically, the value is set to 9 time slices. 
- **MACD Histogram**: It is the difference between the MACD line and the MACD Signal line. A bullish crossover occurs when the MACD line crosses above the MACD Signal line. A bearish crossover occurs if the MACD line crosses below the MACD Signal line. 
- **Open to close performance**: For each time slice, the difference between the closing price and the opening price of that particular period is given as a percentage.
- **High low difference**: The deviation within the current period between the highest and lowest price as a percentage. This shows the volatility within one time slice.
- **Current high versus previous close absolute value**: The absolute value of the difference between the high of the current period and the close value of the previous period (needed for ATR calculation)
- **Current low versus previous close absolute value**: The absolute value of the difference between the low of the current period and the close value of the previous period (needed for ATR calculation)
- **True Range**: The max value of the previous three values (needed for ATR calculation).
- **Average True Range (ATR)**:  This is a is a market volatility indicator used in technical analysis. It takes the moving average of the True Range (typically over a 14 day period). It is not (!) an indicator for price direction. 
- **Absolute performance of last 3 periods**: This indicator provides the performance over the last three time slices as an absolute value.
- **Performance of last 3 periods in percentage**: This indicator provides the performance over the last three time slices as a relative value.
- **Bull or Bear market**: It simply writes "Bull" in case the MACD Histogram is positive and it writes "Bear" in case the MACD Histogram value is negative. 
- **Market trend continued**: To identify a shift from Bull to Bear market and vice versa, this column holds "True" in case the trend from the previous time slice continues and "False" in case a trend change has occurred. 

The enhanced data is stored in a pandas dataframe called df_history_enhanced. 

Learn more about the different trading indicators like SMA, EMA, MACD and ATR, take a look [here](https://www.tradingview.com)

In [12]:
# Set the period for calculating Average True Range (ATR)
AVERAGE_TRUE_RANGE_PERIODS = 14

# Create an empty list to store the enhanced data
currency_history_rows_enhanced = []

# Loop through each currency in MY_CRYPTO_CURRENCIES
for currency in MY_CRYPTO_CURRENCIES:
    # Loop through each granularity (time interval) in GRANULARITIES
    for granularity in GRANULARITIES:
        # Filter the data for the current currency and granularity
        df_history_currency = df_history.query('granularity == @granularity & base_currency == @currency').copy()
        
        # Sort the data by date from oldest to newest (needed for calculations)
        df_history_currency = df_history_currency.sort_values(['date'], ascending=True)
        
        # Calculate Simple Moving Averages (SMA) for 3 and 7 periods
        df_history_currency['SMA3'] = df_history_currency['close'].rolling(window=3).mean()
        df_history_currency['SMA7'] = df_history_currency['close'].rolling(window=7).mean()
        
        # Calculate Exponential Moving Averages (EMA) for 12 and 26 periods
        df_history_currency['EMA12'] = df_history_currency['close'].ewm(span=12, adjust=False).mean()
        df_history_currency['EMA26'] = df_history_currency['close'].ewm(span=26, adjust=False).mean()
        
        # Calculate MACD (Moving Average Convergence Divergence)
        df_history_currency['MACD'] = df_history_currency['EMA12'] - df_history_currency['EMA26']
        
        # Calculate MACD signal line (9-period EMA of MACD)
        df_history_currency['MACD_signal'] = df_history_currency['MACD'].ewm(span=9, adjust=False).mean()
        
        # Calculate MACD histogram (difference between MACD and signal line)
        df_history_currency['macd_histogram'] = df_history_currency['MACD'] - df_history_currency['MACD_signal']
        
        # Calculate open-to-close performance
        df_history_currency['open_to_close_perf'] = ((df_history_currency['close'] - df_history_currency['open']) / df_history_currency['open'])
        
        # Calculate the difference between high and low prices
        df_history_currency['high_low_diff'] = df_history_currency['high'] - df_history_currency['low']
        
        # Calculate the absolute difference between high and previous close
        df_history_currency['high_prev_close_abs'] = np.abs(df_history_currency['high'] - df_history_currency['close'].shift())
        
        # Calculate the absolute difference between low and previous close
        df_history_currency['low_prev_close_abs'] = np.abs(df_history_currency['low'] - df_history_currency['close'].shift())
        
        # Calculate True Range (max of high-low, high-prev close, low-prev close)
        df_history_currency['true_range'] = df_history_currency[['high_low_diff', 'high_prev_close_abs', 'low_prev_close_abs']].max(axis=1)
        
        # Calculate Average True Range using an Exponential Moving Average (EMA)
        df_history_currency['average_true_range'] = df_history_currency['true_range'].ewm(alpha=1/AVERAGE_TRUE_RANGE_PERIODS, min_periods=AVERAGE_TRUE_RANGE_PERIODS, adjust=False).mean()
        
        # Calculate the performance of high-low difference relative to high price
        df_history_currency['high_low_perf'] = (df_history_currency['high'] - df_history_currency['low']) / df_history_currency['high']
        
        # Calculate the open price performance over the last 3 periods (absolute and percentage change)
        df_history_currency['open_perf_last_3_period_abs'] = df_history_currency['open'].rolling(window=4).apply(lambda x: x.iloc[1] - x.iloc[0])
        df_history_currency['open_perf_last_3_period_per'] = df_history_currency['open'].rolling(window=4).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
        
        # Identify whether the market is bullish or bearish based on MACD histogram
        df_history_currency['bull_bear'] = np.where(df_history_currency['macd_histogram'] < 0, 'Bear', 'Bull')
        
        # Append the enhanced data for the current currency and granularity to the list
        currency_history_rows_enhanced.append(df_history_currency)

# Combine all the enhanced data into a single DataFrame
df_history_enhanced = pd.concat(currency_history_rows_enhanced, ignore_index=True)

# Sort the data by base currency, granularity, and date for trend analysis
df_history_enhanced = df_history_enhanced.sort_values(['base_currency', 'granularity', 'date'], ascending=True)

# Identify if the market trend continued (bullish/bearish) from the previous period
df_history_enhanced['market_trend_continued'] = df_history_enhanced.bull_bear.eq(df_history_enhanced.bull_bear.shift()) & \
                                                df_history_enhanced.base_currency.eq(df_history_enhanced.base_currency.shift()) & \
                                                df_history_enhanced.granularity.eq(df_history_enhanced.granularity.shift())

# Show the first 10 rows of the enhanced DataFrame
df_history_enhanced.head(10)


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,...,high_low_diff,high_prev_close_abs,low_prev_close_abs,true_range,average_true_range,high_low_perf,open_perf_last_3_period_abs,open_perf_last_3_period_per,bull_bear,market_trend_continued
390,1768844700,80001.37,80096.55,80032.87,80056.31,1.861345,BTC,15MIN,EUR,2026-01-19 17:45:00,...,95.18,NaN,NaN,95.18,NaN,0.001188,NaN,NaN,Bull,False
391,1768845600,79912.11,80078.17,80071.42,79918.38,1.187327,BTC,15MIN,EUR,2026-01-19 18:00:00,...,166.06,21.86,144.20,166.06,NaN,0.002074,NaN,NaN,Bear,False
392,1768846500,79801.00,79949.40,79912.47,79809.55,0.936126,BTC,15MIN,EUR,2026-01-19 18:15:00,...,148.40,31.02,117.38,148.40,NaN,0.001856,NaN,NaN,Bear,True
393,1768847400,79768.88,79964.21,79818.36,79933.07,0.923818,BTC,15MIN,EUR,2026-01-19 18:30:00,...,195.33,154.66,40.67,195.33,NaN,0.002443,38.55,0.000482,Bear,True
394,1768848300,79862.08,79944.57,79944.57,79907.40,0.770491,BTC,15MIN,EUR,2026-01-19 18:45:00,...,82.49,11.50,70.99,82.49,NaN,0.001032,-158.95,-0.001985,Bear,True
395,1768849200,79837.57,79947.07,79905.36,79907.22,0.643771,BTC,15MIN,EUR,2026-01-19 19:00:00,...,109.50,39.67,69.83,109.50,NaN,0.001370,-94.11,-0.001178,Bear,True
396,1768850100,79872.61,79977.39,79912.82,79906.48,0.977726,BTC,15MIN,EUR,2026-01-19 19:15:00,...,104.78,70.17,34.61,104.78,NaN,0.001310,126.21,0.001581,Bear,True
397,1768851000,79866.32,79983.82,79902.94,79968.78,0.825940,BTC,15MIN,EUR,2026-01-19 19:30:00,...,117.50,77.34,40.16,117.50,NaN,0.001469,-39.21,-0.000490,Bear,True
398,1768851900,79920.68,79996.88,79980.35,79953.20,0.511727,BTC,15MIN,EUR,2026-01-19 19:45:00,...,76.20,28.10,48.10,76.20,NaN,0.000953,7.46,0.000093,Bear,True
399,1768852800,79950.19,80053.58,79950.19,80014.08,0.780259,BTC,15MIN,EUR,2026-01-19 20:00:00,...,103.39,100.38,3.01,103.39,NaN,0.001292,-9.88,-0.000124,Bull,False


## Step 6 - Print Out Selected Information

Now, we can leverage the data which is available in df_history_enhanced to compile some basic parameters that we might want to use in a personal dashboard down the line.

In [13]:
# Loop through all crypto currencies to gather and process additional market data
additional_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True)  # Sort data from oldest to newest
    row_data = [
        currency,
        # Current value of the cryptocurrency
        float(df_24hstats.query('base_currency == @currency')['last'].to_string(index=False)),
        # Current market trend for 1 minute and previous 1 minute trend
        df_history_enhanced.query('granularity == \'1MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False),
        df_history_enhanced.query('granularity == \'1MIN\' and base_currency == @currency').iloc[-2]['bull_bear'],
        # Market trend for 15 minutes and previous 15 minutes trend
        df_history_enhanced.query('granularity == \'15MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False),
        df_history_enhanced.query('granularity == \'15MIN\' and base_currency == @currency').iloc[-2]['bull_bear'],
        # Market trend for 60 minutes and previous 60 minutes trend
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False),
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').iloc[-2]['bull_bear'],
        # Lowest price in the last 24 hours, 30 days, and 90 days
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(24)['low'].min(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(30)['low'].min(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['low'].min(),
        # Highest price in the last 24 hours, 30 days, and 90 days
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(24)['high'].max(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(30)['high'].max(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['high'].max(),
        # Max high/low difference over the last 90 days and 12 days
        round(df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['high_low_perf'].max(), 3),
        round(df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency')['high_low_perf'].max(), 3)
    ]
    # Add the gathered row data to the list
    additional_info_rows.append(row_data)

# Convert the data into a DataFrame and format numerical values
df_additional_info = pd.DataFrame(additional_info_rows, columns=['base_currency', 'current_val', '1min_trend', '1min_prev_trend', 
                                                                 '15min_trend', '15min_prev_trend', '60min_trend', '60min_prev_trend', 
                                                                 '24h_low', '30day_low', '90day_low', '24h_high', '30day_high', '90day_high', 
                                                                 '90day_max_high_low_span', '12day_max_high_low_span'])

# Formatting the values for display
df_additional_info['current_val'] = df_additional_info['current_val'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['24h_low'] = df_additional_info['24h_low'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['30day_low'] = df_additional_info['30day_low'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['90day_low'] = df_additional_info['90day_low'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['24h_high'] = df_additional_info['24h_high'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['30day_high'] = df_additional_info['30day_high'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['90day_high'] = df_additional_info['90day_high'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['90day_max_high_low_span'] = (df_additional_info['90day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info['12day_max_high_low_span'] = (df_additional_info['12day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))

# Transpose the DataFrame for better viewing
df_additional_info.set_index('base_currency').transpose()

base_currency,BTC,ETH,SOL,XRP
current_val,"76,085.25$","2,493.67$",109.19$,1.63$
1min_trend,Bear,Bear,Bear,Bear
1min_prev_trend,Bear,Bear,Bear,Bear
15min_trend,Bull,Bull,Bull,Bull
15min_prev_trend,Bull,Bull,Bull,Bull
60min_trend,Bear,Bear,Bear,Bear
60min_prev_trend,Bear,Bear,Bear,Bear
24h_low,"75,422.93$","2,477.00$",108.02$,1.62$
30day_low,"73,317.59$","2,447.38$",101.16$,1.54$
90day_low,"70,005.42$","2,278.00$",99.64$,1.51$


## Step 7 - Visualise Data

For visualisation - use plotly.

### Simple Visualisation

First, let's create a simple OHLC (Open, High, Low, Close) candle stick chart for each of our currencies.

In [14]:
#!pip3 install plotly
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)  # Initialize Plotly in offline mode for notebook compatibility
import plotly.graph_objs as go  # Import Plotly graphing objects for creating visualizations

df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True)  # Sort data by date (oldest to newest) for visualization

granularity = '15MIN'  # Set the granularity (time interval) for the chart

# Loop over each cryptocurrency in MY_CRYPTO_CURRENCIES to generate a chart for each
for currency in MY_CRYPTO_CURRENCIES:
    # Filter data for the specific granularity and currency
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'')
    
    fig = go.Figure()  # Create a new figure for the chart

    # Add a Candlestick trace (OHLC chart) for the currency
    fig.add_trace(go.Candlestick(
        x=df_history_for_chart['date'],  # Set the x-axis to be the date
        open=df_history_for_chart['open'],  # Set the open prices
        high=df_history_for_chart['high'],  # Set the high prices
        low=df_history_for_chart['low'],  # Set the low prices
        close=df_history_for_chart['close'],  # Set the close prices
        name='OHLC'  # Name of the trace
    ))

    # Add the Average True Range (ATR) line on a secondary y-axis
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],  # Set the x-axis to be the date
        y=df_history_for_chart['average_true_range'],  # Set the y-values to the ATR
        line=dict(color=MACD_COLOR),  # Set the line color for ATR (should be predefined)
        mode='lines',  # Display the ATR as a line
        name="ATR",  # Name of the ATR trace
        yaxis="y2"  # Attach ATR to the secondary y-axis
    ))

    # Customize candlestick colors based on price movement
    cs = fig.data[0]  # Access the candlestick trace
    cs.increasing.fillcolor = CANDLE_INCREASE_COLOR  # Set color for increasing candles
    cs.increasing.line.color = CANDLE_INCREASE_COLOR  # Set border color for increasing candles
    cs.decreasing.fillcolor = CANDLE_DECREASE_COLOR  # Set color for decreasing candles
    cs.decreasing.line.color = CANDLE_DECREASE_COLOR  # Set border color for decreasing candles

    # Update layout settings for the chart
    fig.update_layout(
        yaxis=dict(
            title=MY_QUOTE_CURRENCY,  # Set the title for the primary y-axis (currency values)
            side="right",  # Place the y-axis on the right side
        ),
        yaxis2=dict(
            title="Average True Range (Volatility)",  # Title for the secondary y-axis (ATR)
            anchor="free",  # Set to free to avoid overlaying with the primary y-axis
            overlaying="y",  # Overlay this y-axis over the primary one
            side="left",  # Place the secondary y-axis on the left
        ),
        yaxis2_showgrid=False,  # Disable grid for the secondary y-axis
        yaxis2_zeroline=False,  # Disable the zero line for the secondary y-axis
        xaxis_rangeslider_visible=False,  # Hide the range slider below the x-axis
        title=f'OHLC Chart for {currency} with time interval {granularity}',  # Set the chart title
        hovermode='x unified',  # Show all data for a specific x-axis point when hovering
        xaxis=dict(
            rangeslider=dict(visible=True),  # Make the range slider visible for zooming
            type="date"  # Set the x-axis to display dates
        ),
        template=PLOTLY_TEMPLATE,  # Apply a custom Plotly template (should be predefined)
    )

    # Display the chart
    fig.show()

### Advanced Visualization

Now that we have drawn a basic OHLC chart, let's get a bit more fancy. We will now draw the chart again. But this time we will add the following information:
- MACD line
- MACD signal line
- Colored area to show if we are currently in a bull market (green) or bear market (red) in each given period

Construct the data needed to color different periods for the market trends (bull versus bear). Create another pandas dataframe df_history_market_trend_intervals.

In [15]:
# Initialize an empty list to store market trend intervals for all cryptocurrencies
market_trend_interval_rows = []

# Loop through each cryptocurrency in the MY_CRYPTO_CURRENCIES list
for currency in MY_CRYPTO_CURRENCIES:
    # Loop through each granularity type in the GRANULARITIES list
    for granularity in GRANULARITIES:
        # Filter data for the specific currency, where market_trend_continued is False and granularity matches
        df_history_market_trend_intervals = df_history_enhanced.query(
            'base_currency == @currency and market_trend_continued == False and granularity == @granularity'
        ).copy()

        # Create a new column 'next_period_date' by shifting the 'date' column by one period forward
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals.date.shift(-1)
        
        # Fill any NaN values in the 'next_period_date' column with the current date and time
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals['next_period_date']
        df_history_market_trend_intervals.next_period_date = df_history_market_trend_intervals.next_period_date.fillna(datetime.now())
        
        # Create a new column 'color' to assign a color based on the 'bull_bear' value (Bull -> GREEN, Bear -> RED)
        df_history_market_trend_intervals['color'] = df_history_market_trend_intervals['bull_bear'].apply(
            lambda x: GREEN_COLOR if x == 'Bull' else RED_COLOR
        )
        
        # Select and rename relevant columns for market trend intervals: base_currency, granularity, bull_bear, color, date, and next_period_date
        df_history_market_trend_intervals = df_history_market_trend_intervals[[
            'base_currency', 'granularity', 'bull_bear', 'color', 'date', 'next_period_date'
        ]].rename(columns={"date": "start_date", "next_period_date": "finish_date"})
        
        # Append the processed DataFrame to the list of market trend interval rows
        market_trend_interval_rows.append(df_history_market_trend_intervals)

# Concatenate all the market trend interval rows into a single DataFrame
df_history_market_trend_intervals = pd.concat(market_trend_interval_rows, ignore_index=True)

In [16]:
# CHANGE HERE: 1MIN, 15MIN, 60MIN, DAILY
granularity = '60MIN'  # Set the desired granularity for the chart (options: '1MIN', '15MIN', '60MIN', 'DAILY')

# Loop through each cryptocurrency in the MY_CRYPTO_CURRENCIES list
for currency in MY_CRYPTO_CURRENCIES:
    # Filter data for the specific currency and granularity from the df_history_enhanced DataFrame
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'').copy()
    
    # Filter data for the specific currency and granularity from the df_history_market_trend_intervals DataFrame
    df_history_market_trend_for_chart = df_history_market_trend_intervals.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'').copy()
    
    # Get the minimum and maximum values of the MACD for the chart's y-axis range
    macd_min = df_history_for_chart['MACD'].min()
    macd_max = df_history_for_chart['MACD'].max()
    
    # Create a figure with secondary y-axis support for multiple traces
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add OHLC (Open, High, Low, Close) data to the chart, including color customization for increasing and decreasing candles
    fig.add_trace(go.Ohlc(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        increasing_line_color=CANDLE_INCREASE_COLOR,  # Set the color for increasing candles
        decreasing_line_color=CANDLE_DECREASE_COLOR,  # Set the color for decreasing candles
        name='OHLC'), secondary_y=True)
    
    # Add MACD (Moving Average Convergence Divergence) data to the chart
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD'],
        name='MACD',
        line=dict(color=MACD_COLOR),  # Set the color for the MACD line
        mode='lines'), secondary_y=False)
    
    # Add MACD Signal data to the chart (the signal line for MACD)
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD_signal'],
        name='MACD Signal',
        line=dict(color=MACD_SIGNAL_COLOR),  # Set the color for the MACD Signal line
        mode='lines'), secondary_y=False)
    
    # Loop through the market trend intervals for the current currency to add vertical rectangles indicating trend periods
    for i in range(df_history_market_trend_for_chart['base_currency'].count()):
        fig.add_vrect(
            x0=df_history_market_trend_for_chart.iloc[i]['start_date'],  # Start of the trend interval
            x1=df_history_market_trend_for_chart.iloc[i]['finish_date'],  # End of the trend interval
            col=1,
            # Optional: You can set the fill color for the rectangle, currently commented out
            line=dict(color=df_history_market_trend_for_chart.iloc[i]['color']),  # Set the line color of the rectangle (based on the trend)
            opacity=1.0,
            line_width=1
        )
    
    # Update the layout of the figure to set various chart properties
    fig.update_layout(
        title=f'OHLC Chart for {currency} with time interval {granularity}',  # Set chart title
        hovermode='x unified',  # Set hover mode to show data across the x-axis
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.94),  # Position the legend
        template=PLOTLY_TEMPLATE,  # Set the plot template (style)
        yaxis_range=[macd_min, macd_max],  # Set the y-axis range based on the MACD data
        yaxis1_showgrid=False,  # Hide grid lines for the primary y-axis
        yaxis1_showticklabels=False,  # Hide tick labels for the primary y-axis
        yaxis1_zeroline=False,  # Hide the zero line for the primary y-axis
        margin=dict(
            l=50,  # Left margin
            r=50,  # Right margin
            b=100,  # Bottom margin
            t=100,  # Top margin
            pad=4  # Padding for the margins
        ),
        xaxis=dict(
            rangeselector=dict(
                buttons=[dict(step="all")]  # Set the range selector to show all data
            ),
            rangeslider=dict(visible=True),  # Add a range slider below the x-axis
            type="date"  # Set the x-axis to represent dates
        )
    )
    
    # Set the x-axis tick format depending on the granularity (time interval)
    if granularity == 'DAILY':
        fig.update_layout(xaxis_tickformat='%d %b (%a)')  # Format the date as day/month (week day)
    else:
        fig.update_layout(xaxis_tickformat='%d %b %H:%M')  # Format the date as day/month hour:minute
    
    # Display the figure
    fig.show()


# BOOM!

TESTING AROUND 

In [17]:
import pandas as pd  # For handling datasets and manipulating tables
import requests  # To make API requests and fetch data from the web
import numpy as np  # For numerical operations and calculations
import plotly.graph_objects as go  # For creating visualizations
from plotly.subplots import make_subplots  # To create multiple plots in one figure

# Function to fetch Coinbase historical data
def fetch_coinbase_data(product_id='BTC-USD', granularity=86400):
    """Fetches historical cryptocurrency data from Coinbase API."""
    url = f'https://api.exchange.coinbase.com/products/{product_id}/candles?granularity={granularity}'  # Construct API endpoint
    response = requests.get(url)  # Send request to Coinbase API
    data = response.json()  # Convert response to JSON format
    
    # Create a DataFrame with labeled columns
    df = pd.DataFrame(data, columns=['timestamp', 'low', 'high', 'open', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')  # Convert timestamp to readable date format
    df = df.sort_values('timestamp')  # Ensure data is in chronological order
    return df  # Return formatted dataset

# Function to calculate ATR (Average True Range)
def calculate_atr(df, period=14):
    """Calculates Average True Range (ATR) to measure market volatility."""
    df['high-low'] = df['high'] - df['low']  # High minus low price
    df['high-prev_close'] = abs(df['high'] - df['close'].shift(1))  # High minus previous close
    df['low-prev_close'] = abs(df['low'] - df['close'].shift(1))  # Low minus previous close
    df['TR'] = df[['high-low', 'high-prev_close', 'low-prev_close']].max(axis=1)  # True range calculation
    df['ATR'] = df['TR'].rolling(window=period).mean()  # Moving average of true range
    return df

# Function to calculate EMA (Exponential Moving Average)
def calculate_ema(df, period=50):
    """Calculates the Exponential Moving Average (EMA) for trend analysis."""
    df[f'EMA_{period}'] = df['close'].ewm(span=period, adjust=False).mean()  # EMA formula
    return df

# Function to calculate RSI (Relative Strength Index)
def calculate_rsi(df, period=14):
    """Calculates the RSI to identify overbought or oversold conditions."""
    delta = df['close'].diff(1)  # Price difference between days
    gain = np.where(delta > 0, delta, 0)  # Keep only gains
    loss = np.where(delta < 0, -delta, 0)  # Keep only losses
    avg_gain = pd.Series(gain).rolling(window=period, min_periods=1).mean()  # Average gain over time
    avg_loss = pd.Series(loss).rolling(window=period, min_periods=1).mean()  # Average loss over time
    rs = avg_gain / avg_loss  # Relative strength calculation
    df['RSI'] = 100 - (100 / (1 + rs))  # RSI formula
    return df

# Function to detect whale activity with improved visualization
def detect_whale_activity(df, volume_threshold=2.0):
    """Detects whale activity based on sudden large volume spikes and flags them in the chart."""
    df['Avg_Volume'] = df['volume'].rolling(window=20).mean()  # Calculate 20-day average volume
    df['Whale_Alert'] = df['volume'] > (df['Avg_Volume'] * volume_threshold)  # Flag whale trades
    return df

# Fetch data and apply calculations
df = fetch_coinbase_data()  # Get price data
df = calculate_atr(df)  # Compute ATR

df = calculate_ema(df, period=50)  # Compute EMA
df = calculate_rsi(df, period=14)  # Compute RSI
df = detect_whale_activity(df)  # Detect whale trades

# Plot results
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# Price chart with EMA
fig.add_trace(go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Price'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['EMA_50'], mode='lines', name='EMA 50', line=dict(color='orange')), row=1, col=1)

# Highlight Whale Activity on Price Chart
whale_data = df[df['Whale_Alert']]  # Filter whale trades
fig.add_trace(go.Scatter(x=whale_data['timestamp'], y=whale_data['close'], mode='markers', marker=dict(size=10, color='blue'), name='Whale Alert'), row=1, col=1)

# ATR plot
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['ATR'], mode='lines', name='ATR', line=dict(color='pink')), row=2, col=1)

# RSI plot
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['RSI'], mode='lines', name='RSI', line=dict(color='purple')), row=3, col=1)

fig.update_layout(title='Coinbase BTC Analysis with ATR, EMA & RSI (Whale Alerts)', xaxis_title='Date', template='plotly_dark', height=900)
fig.show()

# Display detected whale activity
df[df['Whale_Alert']][['timestamp', 'volume', 'Avg_Volume', 'close']]  # Show whale trades in table format


,timestamp,volume,Avg_Volume,close
330,2025-02-26,21568.554631,9312.093070,84111.78
328,2025-02-28,30755.742451,10934.493830,84297.73
295,2025-04-02,16748.282623,7631.956467,82490.08
290,2025-04-07,26706.529308,9361.950202,79140.01
288,2025-04-09,23584.944641,10140.231002,82594.75
259,2025-05-08,15379.435441,7498.741484,103253.49
246,2025-05-21,18333.914913,7209.095722,109699.54
196,2025-07-10,12920.207429,5353.500470,116031.30
192,2025-07-14,13536.799548,5536.930707,119863.71
191,2025-07-15,23726.313561,6369.613567,117781.64
